## Step 1: Download Dataset

Download the Urban8K dataset from the link below:

https://urbansounddataset.weebly.com/download-urbansound8k.html

The page will ask for basic details and then direct to the download link. The data size is about 6GB.

## Step 2: Unzip the Dataset

Once downloaded, unzip the file in "..\data\unzipped\" folder path.


## Step 3: Create a Spectrogram Folder

Create a folder to store spectrogram images for various puposes.

1. Create folder path "..\spectogram_images\train"
2. Create folder path "..\spectogram_images\test"
3. Create folder path "..\spectogram_images\adv_train_in"
4. Create folder path "..\spectogram_images\adv_test_in"
5. Create folder path "..\spectogram_images\adv_train_out"
6. Create folder path "..\spectogram_images\adv_test_out"
7. Create folder path "..\spectrogram_images\train_adv"

In [1]:
from IPython.display import Audio
import librosa
from librosa import display
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import cm
import pylab
from PIL import Image
from matplotlib.pyplot import imshow
import os as os
import pandas as pd
import pickle
from shutil import copy

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils.class_weight import compute_class_weight

import torch
import torch.nn as nn
from torchvision import models
import torchvision
from collections import OrderedDict
import torch.nn.functional as F
from torch import optim
from torchvision import transforms
from torch.autograd import Variable

import progressbar
from time import sleep

%matplotlib inline

In [ ]:
#Preprocessing Files in Batch to generate and save Spectogram images

WAV_DIR = 'data/unzipped/UrbanSound8K/audio/fold'
IMG_DIR = 'spectrogram_images/train/'
I_ADV_TRAIN_DIR = 'spectrogram_images/adv_train_in/'
I_ADV_TEST_DIR = 'spectrogram_images/adv_test_in/'
O_ADV_TRAIN_DIR = 'spectrogram_images/adv_train_out/'
O_ADV_TEST_DIR = 'spectrogram_images/adv_test_out/'

ADV_TRAIN_DIR = 'spectrogram_images/train_adv/'



# for i in range(1,4):
for i in range(1,9):
    wav_files = os.listdir(WAV_DIR + str(i) + '/')
    for f in wav_files:
        try:
            # Read wav-file
            y, sr = librosa.load(WAV_DIR + str(i) + '/' + f, sr = 22050)

            # Compute spectrogram
            M = librosa.feature.melspectrogram(y, sr, fmax = sr/2, n_fft=2048, hop_length=512, n_mels = 96, power = 2)

            # Power in DB
            log_power = librosa.power_to_db(M, ref=np.max)

            # Plotting the spectrogram
            pylab.figure(figsize=(5,5))
            pylab.axis('off') 
            pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[])
            librosa.display.specshow(log_power, cmap=cm.jet)
            pylab.savefig(IMG_DIR + f[:-4]+'.jpg', bbox_inches=None, pad_inches=0)
            pylab.close()

        except Exception as e:
            print(f, e)
            pass

for i in range(9,10):
    wav_files = os.listdir(WAV_DIR + str(i) + '/')
    for f in wav_files:
        try:
            # Read wav-file
            y, sr = librosa.load(WAV_DIR + str(i) + '/' + f, sr = 22050)

            # Compute spectrogram
            M = librosa.feature.melspectrogram(y, sr, fmax = sr/2, n_fft=2048, hop_length=512, n_mels = 96, power = 2)

            # Power in DB
            log_power = librosa.power_to_db(M, ref=np.max)

            # Plotting the spectrogram
            pylab.figure(figsize=(5,5))
            pylab.axis('off') 
            pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[])
            librosa.display.specshow(log_power, cmap=cm.jet)
            pylab.savefig(I_ADV_TRAIN_DIR + f[:-4]+'.jpg', bbox_inches=None, pad_inches=0)
            pylab.close()

        except Exception as e:
            print(f, e)
            pass
        
for i in range(10,11):
    wav_files = os.listdir(WAV_DIR + str(i) + '/')
    for f in wav_files:
        try:
            # Read wav-file
            y, sr = librosa.load(WAV_DIR + str(i) + '/' + f, sr = 22050)

            # Compute spectrogram
            M = librosa.feature.melspectrogram(y, sr, fmax = sr/2, n_fft=2048, hop_length=512, n_mels = 96, power = 2)

            # Power in DB
            log_power = librosa.power_to_db(M, ref=np.max)

            # Plotting the spectrogram
            pylab.figure(figsize=(5,5))
            pylab.axis('off') 
            pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[])
            librosa.display.specshow(log_power, cmap=cm.jet)
            pylab.savefig(I_ADV_TEST_DIR + f[:-4]+'.jpg', bbox_inches=None, pad_inches=0)
            pylab.close()

        except Exception as e:
            print(f, e)
            pass

.DS_Store 
.DS_Store 
.DS_Store 
.DS_Store 
.DS_Store 


In [4]:
IMG_DIR = 'spectrogram_images/train/'
IMG_HEIGHT = 224
IMG_WIDTH = 224
NUM_CLASSES = 10
NUM_EPOCHS = 16
BATCH_SIZE = 10
L2_LAMBDA = 0.001
IMG_SIZE = (224, 224)
WAV_DIR = 'data/unzipped/UrbanSound8K/audio/fold'
IMG_DIR = 'spectrogram_images/train/'
I_ADV_TRAIN_DIR = 'spectrogram_images/adv_train_in/'
I_ADV_TEST_DIR = 'spectrogram_images/adv_test_in/'
O_ADV_TRAIN_DIR = 'spectrogram_images/adv_train_out/'
O_ADV_TEST_DIR = 'spectrogram_images/adv_test_out/'

In [5]:
one_hot = OneHotEncoder(categories=[range(NUM_CLASSES)])

all_files = os.listdir(IMG_DIR)

# Get class weights
label_array = []
for file_ in all_files:
    vals = file_.split('-')[1]
    label_array.append(int(vals))
    
cl_weight = compute_class_weight(class_weight = 'balanced', 
                                 classes = np.unique(label_array), 
                                 y = label_array)

# Train-val-test split of files
train_files, test_files, train_labels, test_labels = train_test_split(all_files, 
                                                                      label_array,
                                                                      random_state = 10, 
                                                                      test_size = 0.2
                                                                     )

# Among the test files, keep half for validation
val_files, test_files, val_labels, test_labels = train_test_split(test_files, test_labels,
                                                                  random_state = 10, 
                                                                  test_size = 0.5
                                                                 )

adv_train_files = os.listdir(I_ADV_TRAIN_DIR)

# Get class weights
adv_train_label = []
for file_ in all_files:
    vals = file_.split('-')[1]
    adv_train_label.append(int(vals))

adv_test_files = os.listdir(I_ADV_TEST_DIR)

# Get class weights
adv_test_label = []
for file_ in all_files:
    vals = file_.split('-')[1]
    adv_test_label.append(int(vals))

In [6]:
loader = transforms.Compose([transforms.Resize(IMG_SIZE), transforms.ToTensor()])
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

def image_loader(image_name):
    """load image, returns tensor"""
    image = Image.open(image_name)
    image = image.convert("RGB")
    image = loader(image).float()
    image = normalize(image).float()
    return image


def load_batch(file_list):
    img_array = []
    idx_array = []
    label_array = []

    for file_ in file_list:
        im = IMG_DIR + file_
        
        img_array.append(np.array(image_loader(im)))

        vals = file_.split('-')
        idx_array.append(vals[0])
        label_array.append([int(vals[1])])

    label_array = one_hot.fit_transform(label_array).toarray()
    img_array = torch.FloatTensor(img_array)
    label_array = torch.LongTensor(label_array)
    
    return img_array, label_array

def batch_generator(files, BATCH_SIZE):
    L = len(files)

    #this line is just to make the generator infinite, keras needs that    
    while True:

        batch_start = 0
        batch_end = BATCH_SIZE

        while batch_start < L:
            
            limit = min(batch_end, L)
            file_list = files[batch_start: limit]
            batch_img_array, batch_label_array = load_batch(file_list)

            yield (batch_img_array, batch_label_array) # a tuple with two numpy arrays with batch_size samples     

            batch_start += BATCH_SIZE   
            batch_end += BATCH_SIZE

In [7]:
#Model Architecture Definition

model = models.vgg16(pretrained=True) #load vgg model

# you need to flatten the layer after vgg16 to get 18432, check the keras model in the original nb
model.classifier = nn.Sequential(nn.Linear(512*7*7,512),
                                nn.ReLU(),
                                nn.Dropout(0.3),
                                nn.Linear(512,NUM_CLASSES))  
# note there is no softmax layer while training, validating and testing put in that operation manually as the learning
# doesn't depend upon softmax layer, this way we will get logits. We can use logits in fgsm

print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [8]:
#Training and Validation Functions

def train (model, loader, criterion):
    #Custom Code for Progress Bar
    bar = progressbar.ProgressBar(maxval=100, \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()
    
    model.train()
    current_loss = 0
    current_correct = 0
    i = 0
    i_percentage = 0
    for train, y_train in iter(loader):
        i = i+train.shape[0]
        if i > len(train_files):
            break
        y_train = torch.argmax(y_train, dim=1)
        train, y_train = train.to(device), y_train.to(device)
        optimizer.zero_grad()
        output = model.forward(train)
        _, preds = torch.max(output,1)
        loss = criterion(output, y_train)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.classifier.parameters(), 50)
        optimizer.step()
        current_loss += loss.item()*train.size(0)
        current_correct += torch.sum(preds == y_train.data)
        i_percentage = (i/len(train_files))*100
        bar.update(i_percentage)
        
    bar.finish()
    epoch_loss = current_loss / len(train_files)
    epoch_acc = current_correct.double() / len(train_files)
        
    return epoch_loss, epoch_acc

#validate the model
def validation (model, loader, criterion):
    #Custom Code for Progress Bar
    bar = progressbar.ProgressBar(maxval=100, \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()
    
    model.eval()
    valid_loss = 0
    valid_correct = 0
    i = 0
    i_percentage = 0
    for valid, y_valid in iter(loader):
        i = i+valid.shape[0]
        if i > len(val_files):
            break
        y_valid = torch.argmax(y_valid, dim=1)
        valid, y_valid = valid.to(device), y_valid.to(device)
        output = model.forward(valid)
        valid_loss += criterion(output, y_valid).item()*valid.size(0)
        equal = (output.max(dim=1)[1] == y_valid.data)
        valid_correct += torch.sum(equal)#type(torch.FloatTensor)
        i_percentage = (i/len(val_files))*100
        bar.update(i_percentage)
        
    bar.finish()
    epoch_loss = valid_loss / len(val_files)
    epoch_acc = valid_correct.double() / len(val_files)
    
    return epoch_loss, epoch_acc

def testing (model, loader, criterion, test_size):
    #Custom Code for Progress Bar
    bar = progressbar.ProgressBar(maxval=100, \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()
    
    model.eval()
    test_loss = 0
    test_correct = 0
    i = 0
    i_percentage = 0
    for test, y_test in iter(loader):
        i = i+test.shape[0]
        if i > len(test_files):
            break
        y_test = torch.argmax(y_test, dim=1)
        test, y_test = test.to(device), y_test.to(device)
        output = model.forward(test)
        test_loss += criterion(output, y_test).item()*test.size(0)
        equal = (output.max(dim=1)[1] == y_test.data)
        test_correct += torch.sum(equal)#type(torch.FloatTensor)
        i_percentage = (i/test_size)*100
        bar.update(i_percentage)
        
    bar.finish()
    epoch_loss = test_loss / test_size
    epoch_acc = test_correct.double() / test_size
    
    return epoch_loss, epoch_acc

In [10]:
criteria = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.classifier.parameters(), lr = 0.005, momentum = 0.5)
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [14]:
import time
#Train the model

#freeze gradient parameters in pretrained model
for param in model.parameters():
    param.require_grad = True    

#freeze gradient parameters in pretrained model
for param in model.features.parameters():
    param.require_grad = False
    
#train and validate
#4 epochs seem to be enough
epochs = 4
epoch = 0

start = time.clock()    
for e in range(epochs):
    epoch +=1
    with torch.set_grad_enabled(True):
        epoch_train_loss, epoch_train_acc = train(model,batch_generator(train_files,BATCH_SIZE),criteria)
    print("Epoch: {} Train Loss : {:.4f}  Train Accuracy: {:.4f}".format(epoch,epoch_train_loss,epoch_train_acc))
    with torch.no_grad():
        epoch_val_loss, epoch_val_acc = validation(model,batch_generator(val_files,BATCH_SIZE),criteria)
    print("Epoch: {} Validation Loss : {:.4f}  Validation Accuracy {:.4f}".format(epoch,epoch_val_loss,epoch_val_acc))
print(start - time.clock())    

[========================================================================] 100%


Epoch: 1 Train Loss : 0.0334  Train Accuracy: 0.9951


[========================================================================] 100%


Epoch: 1 Validation Loss : 0.3878  Validation Accuracy 0.8941


[========================================================================] 100%


...

[========================================================================] 100%


Epoch: 15 Validation Loss : 0.3552  Validation Accuracy 0.9167


[========================================================================] 100%


Epoch: 16 Train Loss : 0.0040  Train Accuracy: 0.9995


Epoch: 16 Validation Loss : 0.3546  Validation Accuracy 0.9195
-4408.3645240000005


In [15]:
#Save the original training model
torch.save(model,'original_model.pt')

In [16]:
#model = torch.load('original_model.pt',map_location='cpu')

In [17]:
#Test Results

with torch.no_grad():
    epoch_test_loss, epoch_test_acc = testing(model,batch_generator(test_files,BATCH_SIZE),criteria,len(test_files))
print("Epoch: {} Test Loss : {:.4f}  Test Accuracy {:.4f}".format(1,epoch_test_loss,epoch_test_acc))


Epoch: 1 Test Loss : 0.4645  Test Accuracy 0.8785


In [18]:
def load_batch_2(file_list):
    img_array = []
    idx_array = []
    label_array = []

    for file_ in file_list:
        im = I_ADV_TRAIN_DIR + file_
        
        img_array.append(np.array(image_loader(im)))

        vals = file_.split('-')
        idx_array.append(vals[0])
        label_array.append([int(vals[1])])

    label_array = one_hot.fit_transform(label_array).toarray()
    img_array = torch.FloatTensor(img_array)
    label_array = torch.LongTensor(label_array)
    
    return img_array, label_array


#Generate Adversarial Images
batch_size = 5
pred = []
adv_pred = []
if 'adv_x' in locals():
    del adv_x

i=0
bar = progressbar.ProgressBar(maxval=100, \
widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
bar.start()

un_normalize = transforms.Normalize(mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],std=[1/0.229, 1/0.224, 1/0.225])
un_convert = transforms.ToPILImage()

# ADV_IMG_DIR = 'spectrogram_images/adv/'

start = time.time()
adv_size = len(adv_train_files)
file_idx = 0
while i<adv_size:
    j = i + batch_size
    if j>adv_size:
        j = adv_size
    x,y = load_batch_2(adv_train_files[i:j])
    x = Variable(x, requires_grad=True)
    x,y = x.to(device), y.to(device)
    
    output = model.forward(x)
    y_label = torch.argmax(y, dim=1)
    y_label = y_label.to(device)
    loss = criteria(output, y_label)
    loss.backward()

    # Add perturbation 
    epsilon = 0.01
    x_grad     = torch.sign(x.grad.data)
    temp = x.data + epsilon*x_grad
    if 'adv_x' in locals():
        adv_x = torch.cat((adv_x,temp),0)
    else:
        adv_x = temp
    adv_x = adv_x.to(device)
    adv_output = model.forward(adv_x)
    adv_output = adv_output.to(device)

    x_pred = torch.argmax(y, dim=1)
    x_adv_pred = torch.argmax(adv_output,dim=1)
    
    x_pred, x_adv_pred = x_pred.to(device), x_adv_pred.to(device)

    pred.extend(x_pred.tolist())
    adv_pred.extend(x_adv_pred.tolist())
    for ii in range(j-i):
        temp = adv_x[ii]
        t1 = un_normalize(temp).float()
        t2 = un_convert(t1)
        t2.save(O_ADV_TRAIN_DIR + adv_train_files[file_idx], "JPEG", quality=80, optimize=True, progressive=True)
        file_idx += 1
        del temp
        del t1
        del t2
    i = j
    del adv_x
    bar.update((i/adv_size)*100)

bar.finish()

comp = []
for i in range(len(pred)):
    if pred[i] == adv_pred[i]:
        comp.extend([1])
    else:
        comp.extend([0])

adv_acc = (sum(comp)/len(comp))*100
print("The accuracy of model with adversarial examples is ","{0:.2f}".format(adv_acc),"%")
print(start - time.time())

FileNotFoundError: [Errno 2] No such file or directory: 'spectrogram_images/train/103249-5-0-15.jpg'

In [ ]:
#Copy the files from training and adversarial images folder into a single training folder
ADV_TRAIN_DIR = 'spectrogram_images/train_adv/'

for f in train_files:
    copy(IMG_DIR + f,ADV_TRAIN_DIR)

for f in adv_train_files:
    copy(O_ADV_TRAIN_DIR + f,ADV_TRAIN_DIR)




In [ ]:
import gc
gc.collect()

In [ ]:
#Train the network now with original + adversarial images
adv_files = os.listdir(ADV_TRAIN_DIR)

# Get class weights
adv_label = []
for file_ in all_files:
    vals = file_.split('-')[1]
    adv_label.append(int(vals))
    
cl_weight = compute_class_weight(class_weight = 'balanced', 
                                 classes = np.unique(label_array), 
                                 y = label_array)

# Train-val-test split of files
train_files, test_files, train_labels, test_labels = train_test_split(adv_files, 
                                                                      adv_label,
                                                                      random_state = 10, 
                                                                      test_size = 0.2
                                                                     )

# Among the test files, keep half for validation
val_files, test_files, val_labels, test_labels = train_test_split(test_files, test_labels,
                                                                  random_state = 10, 
                                                                  test_size = 0.5
                                                                 )



In [ ]:
def adv_train (model, loader, criterion):
    #Custom Code for Progress Bar
    bar = progressbar.ProgressBar(maxval=100, \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()
    
    model.train()
    current_loss = 0
    current_correct = 0
    i = 0
    i_percentage = 0
    for train, y_train in iter(loader):
        train = Variable(train, requires_grad=True)
        i = i+train.shape[0]
        if i > len(train_files):
            break
        y_train = torch.argmax(y_train, dim=1)
        train, y_train = train.to(device), y_train.to(device)
        optimizer.zero_grad()
        output = model.forward(train)
        _, preds = torch.max(output,1)
        loss = criterion(output, y_train)
        loss.backward(retain_graph=True)
        
#         #adversarial
        
#         output = model.forward(x)
#         y_label = torch.argmax(y, dim=1)
#         loss = criteria(output, y_label)
#         loss.backward()

        # Add perturbation 
        epsilon = 0.01
        train_grad = torch.sign(train.grad.data)
        adv_train = train.data + epsilon*train_grad
        adv_train = adv_train.to(device)
        
        optimizer.zero_grad()
        adv_output = model.forward(adv_train)
        adv_output = adv_output.to(device)
        _, preds = torch.max(adv_output,1)
        loss = 0.6 * (loss) + 0.4 * (criterion(adv_output, y_train))
        loss.backward()
        
        
        torch.nn.utils.clip_grad_norm_(model.classifier.parameters(), 50)
        optimizer.step()
        current_loss += loss.item()*train.size(0)
        current_correct += torch.sum(preds == y_train.data)
        i_percentage = (i/len(train_files))*100
        bar.update(i_percentage)
        
    bar.finish()
    epoch_loss = current_loss / len(train_files)
    epoch_acc = current_correct.double() / len(train_files)
        
    return epoch_loss, epoch_acc

In [ ]:
#Model Architecture Definition

model = models.vgg16(pretrained=True) #load vgg model

# you need to flatten the layer after vgg16 to get 18432, check the keras model in the original nb
model.classifier = nn.Sequential(nn.Linear(512*7*7,512),
                                nn.ReLU(),
                                nn.Dropout(0.3),
                                nn.Linear(512,NUM_CLASSES))  
# note there is no softmax layer while training, validating and testing put in that operation manually as the learning
# doesn't depend upon softmax layer, this way we will get logits. We can use logits in fgsm

criteria = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.classifier.parameters(), lr = 0.005, momentum = 0.5)
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

#freeze gradient parameters in pretrained model
for param in model.parameters():
    param.require_grad = True    

#freeze gradient parameters in pretrained model
for param in model.features.parameters():
    param.require_grad = False



In [ ]:
def load_batch_v(file_list):
    img_array = []
    idx_array = []
    label_array = []

    for file_ in file_list:
        im = ADV_TRAIN_DIR + file_
        
        img_array.append(np.array(image_loader(im)))

        vals = file_.split('-')
        idx_array.append(vals[0])
        label_array.append([int(vals[1])])

    label_array = one_hot.fit_transform(label_array).toarray()
    img_array = torch.FloatTensor(img_array)
    label_array = torch.LongTensor(label_array)
    
    return img_array, label_array

def batch_generator_v(files, BATCH_SIZE):
    L = len(files)

    #this line is just to make the generator infinite, keras needs that    
    while True:

        batch_start = 0
        batch_end = BATCH_SIZE

        while batch_start < L:
            
            limit = min(batch_end, L)
            file_list = files[batch_start: limit]
            batch_img_array, batch_label_array = load_batch(file_list)

            yield (batch_img_array, batch_label_array) # a tuple with two numpy arrays with batch_size samples     

            batch_start += BATCH_SIZE   
            batch_end += BATCH_SIZE

In [ ]:
#train and validate
#Need to see at what epoch gain stops
epochs = 5
epoch = 0

start = time.time()
for e in range(epochs):
    epoch +=1
    print(epoch)
    with torch.set_grad_enabled(True):
        epoch_train_loss, epoch_train_acc = adv_train(model,batch_generator_v(train_files,BATCH_SIZE),criteria)
    print("Epoch: {} Train Loss : {:.4f}  Train Accuracy: {:.4f}".format(epoch,epoch_train_loss,epoch_train_acc))
    with torch.no_grad():
        epoch_val_loss, epoch_val_acc = validation(model,batch_generator_v(val_files,BATCH_SIZE),criteria)
    print("Epoch: {} Validation Loss : {:.4f}  Validation Accuracy {:.4f}".format(epoch,epoch_val_loss,epoch_val_acc))
print(start - time.time())    

In [ ]:
torch.save(model,'adv_model.pt')

In [ ]:
model_adv = torch.load('adv_model.pt',map_location='cpu')

In [ ]:
#Test Results

with torch.no_grad():
    epoch_test_loss, epoch_test_acc = testing(model,batch_generator_v(test_files,BATCH_SIZE),criteria,len(test_files))
print("Epoch: {} Test Loss : {:.4f}  Test Accuracy {:.4f}".format(1,epoch_test_loss,epoch_test_acc))

In [ ]:
def load_batch_3(file_list):
    img_array = []
    idx_array = []
    label_array = []

    for file_ in file_list:
        im = I_ADV_TEST_DIR + file_
        
        img_array.append(np.array(image_loader(im)))

        vals = file_.split('-')
        idx_array.append(vals[0])
        label_array.append([int(vals[1])])

    label_array = one_hot.fit_transform(label_array).toarray()
    img_array = torch.FloatTensor(img_array)
    label_array = torch.LongTensor(label_array)
    
    return img_array, label_array

start = time.time()
#Generate Adversarial Images
batch_size = 5
pred = []
adv_pred = []
if 'adv_x' in locals():
    del adv_x

i=0
bar = progressbar.ProgressBar(maxval=100, \
widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
bar.start()

un_normalize = transforms.Normalize(mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],std=[1/0.229, 1/0.224, 1/0.225])
un_convert = transforms.ToPILImage()

# ADV_IMG_DIR = 'spectrogram_images/adv/'


adv_size = len(adv_test_files)
file_idx = 0
while i<adv_size:
    j = i + batch_size
    if j>adv_size:
        j = adv_size
    x,y = load_batch_3(adv_test_files[i:j])
    x = Variable(x, requires_grad=True)
    x = x.to(device)
    y = y.to(device)
    output = model.forward(x)
    y_label = torch.argmax(y, dim=1)
    y_label = y_label.to(device)
    loss = criteria(output, y_label)
    loss.backward()

    # Add perturbation 
    epsilon = 0.01
    x_grad     = torch.sign(x.grad.data)
    x_grad = x_grad.to(device)
    temp = x.data + epsilon*x_grad
    if 'adv_x' in locals():
        adv_x = torch.cat((adv_x,temp),0)
    else:
        adv_x = temp
    adv_x = adv_x.to(device)
    adv_output = model.forward(adv_x)
    adv_output = adv_output.to(device)

    x_pred = torch.argmax(y, dim=1)
    x_adv_pred = torch.argmax(adv_output,dim=1)
    
    x_pred, x_adv_pred = x_pred.to(device), x_adv_pred.to(device)

    pred.extend(x_pred.tolist())
    adv_pred.extend(x_adv_pred.tolist())
    for ii in range(j-i):
        temp = adv_x[ii]
        t1 = un_normalize(temp).float()
        t2 = un_convert(t1)
        t2.save(O_ADV_TEST_DIR + adv_test_files[file_idx], "JPEG", quality=80, optimize=True, progressive=True)
        file_idx += 1
        del temp
        del t1
        del t2
    i = j
    del adv_x
    bar.update((i/adv_size)*100)

bar.finish()

comp = []
for i in range(len(pred)):
    if pred[i] == adv_pred[i]:
        comp.extend([1])
    else:
        comp.extend([0])

adv_acc = (sum(comp)/len(comp))*100
print("The accuracy of model with adversarial examples is ","{0:.2f}".format(adv_acc),"%")
print(start - time.time()) 